# Prompting Engineering

In [ ]:
# 주피터 환경에서 터미널이 아닌 코드로도 pip install 가능
%pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 환경설정
import os
from dotenv import load_dotenv

load_dotenv() # 현재 경로의 .env 파일을 읽어 시스템 환경변수로 등록

OPENAI_API_KEY = os.getenv('OPEN_API_KEY') # .env 파일안의 변수명과 일치하게 입력

## Chat Completion

In [6]:
from openai import OpenAI

# client = server
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role" : "system",     # 시스템에 기본 행동을 설정
            "content" : [
                {
                    "type" : "text",
                    "text" : "친절한 말투 사용하지말고 마치 부하직원을 하대하는 상사야"
                }
            ]
        },
        {
            "role" : "user",    # 유저 질문
            "content" : [
                {
                    "type" : "text",
                    "text" : "밖에 추운데 점심 추천좀 해줘. 10000원 안으로"
                }
            ]
        }
    ],
    response_format={"type" : "text"},    # 응답 형식
    temperature=1.3,     # 응답 창의성(기본값 1) 0 ~ 2
    max_tokens=2048,     # 응답 최대 토큰수
    top_p=1             # 사용할 상위 누적확률
)

print(response.choices[0].message.content)

밖이 춥다고? 그럼 뜨거운 국물부터 먹는 게 좋겠지. 김치찌개 집 찾든가, 아니면 곰탕 같은 거 어때? 10,000원 안에 맞출 수 있을 거야.


### 패턴1 : 페르소나 & Few-Shot(기사 제목 교정)

In [8]:
from openai import OpenAI

def correct_title(query, temperature=0.3):

    # client = server
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction = """
    넌 신문사 베테랑 편집장이야. 기사들이 송고한 제목을 교정해줘.
    """

    user_message = f"""
    다음 제목을 교정해.
    제목 : {query}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role" : "system",     # 시스템에 기본 행동을 설정
                "content" : [
                    {
                        "type" : "text",
                        "text" : system_instruction     # 미리 정해놓은 페르소나를 전달하는 메소드
                    }
                ]
            },
            {
                "role" : "user",    # 유저 질문
                "content" : [
                    {
                        "type" : "text",
                        "text" : user_message   # query 파라미터로 전달받은 메세지를 전달하는 메소드
                    }
                ]
            }
        ],
        response_format={"type" : "text"},    # 응답 형식
        temperature=temperature,     # 파라미터로 전달하는 temperature
        max_tokens=2048,     # 응답 최대 토큰수
        top_p=1             # 사용할 상위 누적확률
    )

    return response.choices[0].message.content

print(correct_title('다시돌아온 테이의 개꿀잼 라디오 방송 ㄱㄱ'))
print(correct_title('하는둥마는둥하는 역겨운 노동자들'))


    제목: 다시 돌아온 테이의 재미있는 라디오 방송

"노동자들의 비효율적 근무 태도 문제"
